In [4]:
import requests
import json
import pandas as pd
import psycopg2
%run 'C:\\dev\\prod\\metrikaHelper.py'

In [56]:
def getVipProjectByDomain(domain):
    conn = psycopg2.connect("dbname='CRM' user='read_only' host='192.168.10.10' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT p.projectid, p.domain, p2.login, p2.password FROM project AS p INNER JOIN ProjectMetrikaData AS p2 ON p.UniqueId = p2.ProjectUniqueId WHERE domain=%s ORDER BY p.projectid", [domain])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getVipProjectByRow(row))
    return result
def getVipProjectByRow(row):
    return {'projectid': row[0], 'domain': row[1], 'metrikalogin': row[2], 'metrikapassword': row[3]}
def getCPATargetsByProjectID(projectID):
    conn = psycopg2.connect("dbname='Cubo' user='read_only' host='192.168.10.8' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT targetstatus, targetid, targetname FROM metricstarget WHERE projectid=%s ORDER BY id DESC", [projectID])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getCPATargetByRow(row))
    return result
def getCPATargetByRow(row):
    return {'targetstatus': row[0], 'targetid': row[1], 'targetname': row[2]}
def createMetrikaRequest(date1,date2,token,counterID):
    url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?'
    goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(VipProjectInfo['projectid'])])
    otherMetrics = 'ym:s:visits'
    metrics = ','.join([otherMetrics,goalsMetrics])
    payload = {
        'dimensions':'ym:s:gender,ym:s:ageInterval',
        'metrics':metrics,
        'date1':date1,
        'date2':date2,
        'oauth_token':token,
        'ids':counterID,
        'limit':100000,
        'offset':1,
        'filters':"ym:s:UTMMedium=='direct'",
        'accuracy':'full'}
    payloadString = '&'.join(['{0}={1}'.format(k,v) for k,v in payload.iteritems()])
    return '{0}{1}'.format(url,payloadString)

In [22]:
project = u'mebmetall.ru'
VipProjectInfoList = getVipProjectByDomain(project)
VipProjectInfo = (v for v in VipProjectInfoList if v['metrikapassword'] != None).next()
metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':VipProjectInfo['metrikalogin'],'yandexPassword':VipProjectInfo['metrikapassword'],'domain':VipProjectInfo['domain']})

In [57]:
url = createMetrikaRequest('2016-01-01','2016-05-16',metrikaCredentials['Token'],metrikaCredentials['counterID'][0])
df = pd.read_csv(url,encoding='utf-8')
df.rename(columns={u'﻿"Пол"': 'Gender',
                        u'Визиты': 'Visits',
                   u'Возраст': 'Age'
                       }, inplace=True)
for i in xrange(0,len(df)):
    df.loc[i,'Goal Visits'] = df.loc[i,[column for column in df.columns if u'Целевые визиты' in column]].sum()
df.drop([column for column in df.columns if u'Целевые визиты' in column],axis=1,inplace=True)
df['Domain'] = VipProjectInfo['domain']
df = df[~df[u'Gender'].isin([u'Итого и средние'])]
# df = df[~df[u'Кампания Яндекс.Директа'].isin([u'Не определено'])]

In [51]:
df.to_excel(u'C:\\Users\\aleksey.makarov\\Desktop\\2016-05\\2016-05-17 Половозрастной таргетинг\\MetrikaStats.xlsx')

In [58]:
df

,Gender,Age,Visits,Goal Visits,Domain
1,мужской,25‑34 года,1085,1,mebmetall.ru
2,мужской,45 лет и старше,941,0,mebmetall.ru
3,мужской,35‑44 года,651,0,mebmetall.ru
4,женский,25‑34 года,305,1,mebmetall.ru
5,женский,35‑44 года,276,1,mebmetall.ru
6,женский,45 лет и старше,271,0,mebmetall.ru
7,мужской,18‑24 года,242,0,mebmetall.ru
8,женский,18‑24 года,90,0,mebmetall.ru
9,мужской,младше 18 лет,61,0,mebmetall.ru
10,женский,младше 18 лет,41,0,mebmetall.ru
